## [01_Deterministic.ipynb](https://github.com/raybellwaves/xskillscore-tutorial/blob/master/01_Determinisitic.ipynb)

 - In this notebook I show how `xskillscore` can be dropped in a typical data science task where the data is a [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

 - I use the metric RMSE to verifity forecasts of items sold.

 - I also show how you can applys weights to the verification and handle missing values.

Import the neccessary packages

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import xskillscore as xs

Let's say you are a data scientist who works for a company which owns four stores which each sell three items (Store Keeping Units).

Set up `stores` and `skus` arrays:

In [2]:
stores = np.arange(4)
skus = np.arange(3)

and you are tracking daily perfomane of items sold between Jan 1st and Jan 5th 2020.

Setup up `dates` array:

In [3]:
dates = pd.date_range("1/1/2020", "1/5/2020", freq="D")

Generate a `pandas.DataFrame` to show the number of items that were sold during this period. The number of items sold will be a random number between 1 and 10.

This may be something you would obtain from querying a database:

In [4]:
rows = []
for _, date in enumerate(dates):
    for _, store in enumerate(stores):
        for _, sku in enumerate(skus):
            rows.append(
                dict(
                    {
                        "DATE": date,
                        "STORE": store,
                        "SKU": sku,
                        "QUANTITY_SOLD": np.random.randint(9) + 1,
                    }
                )
            )
df = pd.DataFrame(rows)

Pring the first 5 rows of the `pandas.DataFrame`:

In [5]:
df.head()

,DATE,STORE,SKU,QUANTITY_SOLD
0,2020-01-01,0,0,9
1,2020-01-01,0,1,2
2,2020-01-01,0,2,2
3,2020-01-01,1,0,3
4,2020-01-01,1,1,1


Your boss has asked you to use this data to predict the number of items sold for each store and sku level for the next 5 days.

The prediction is outside of the scope of the tutorial but we will use `xskillscore` to tell us how good our prediction may be .

First, rename the target variable to ``y``:

In [6]:
df.rename(columns={"QUANTITY_SOLD": "y"}, inplace=True)
df.head()

,DATE,STORE,SKU,y
0,2020-01-01,0,0,9
1,2020-01-01,0,1,2
2,2020-01-01,0,2,2
3,2020-01-01,1,0,3
4,2020-01-01,1,1,1


Use [pandas MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) to help handle the granularity of the forecast:

In [7]:
df.set_index(['DATE', 'STORE', 'SKU'], inplace=True)

This also displays the data in a cleaner foremat in the notebook:

In [8]:
df.head()

y
DATE       STORE SKU   
2020-01-01 0     0    9
                 1    2
                 2    2
           1     0    3
                 1    1

Time for your prediction! As mentioned, this is outside of the scope of this tutorial.

In our case we are going to generate data to mimic a prediction by taking `y` and perturbing randomly. This will provide a middle ground of creating a prediction which is not overfitting the data (being very similar to `y`) and the other extereme of random numbers for which the skill will be 0.

The pertubations will scale `y` between -100% and 100% using a uniform distribution. For example, a value of 5 in `y` will be between 0 and 10 in the prediction (`yhat`).

Setup the perturbation array:

In [9]:
noise = np.random.uniform(-1, 1, size=len(df['y']))

Name the prediction `yhat` and append it to the `pandas.DataFrame`.

Lastly, convert it is an `int` to match the same format as the target (`y`):

In [10]:
df['yhat'] = (df['y'] + (df['y'] * noise)).astype(int)
df.head()

y  yhat
DATE       STORE SKU         
2020-01-01 0     0    9    13
                 1    2     3
                 2    2     2
           1     0    3     4
                 1    1     0

## Using xskillscore - RMSE

RMSE (root-mean-squre error) is the square root of the average of the squared differences between forecasts and verification data:

\begin{align}
RMSE = \sqrt{\overline{(f - o)^{2}}}
\end{align}

Because the error is squared is it sensitive to outliers and is a more conversative metric than mean-absolute error.

See https://climpred.readthedocs.io/en/stable/metrics.html#root-mean-square-error-rmse for further documentation

### sklearn

Most data scientists are familar with using `scikit-learn` for verifying forecasts, especially if you used `scikit-learn` for the prediction.

To obtain RMSE from `scikit-learn` import `mean_squared_error` and specify `squared=False`:

In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['y'], df['yhat'], squared=False)

2.932575659723036

While `skikit-learn` is simple it doesn't give the flexibility of that given in `xskillscore`.

Note: `xskillscore` does use the same metrics as in `scikit-learn` such as the [`r2_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html), which is called `r2` in `xskillscore`.

### xskillscore

To use `xskillscore` you first have to put your data into an `xarray` object.

Because `xarray` is part of the PyData stack it integrates will other Python data science packages.

`pandas` has a convient [`to_xarray`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_xarray.html) which makes going from `pandas` to `xarray` seemless.

Use `to_xarray` to convert the `pandas.Dataframe` to an `xarray.Dataset`: 

In [12]:
ds = df.to_xarray()
ds

<xarray.Dataset>
Dimensions:  (DATE: 5, SKU: 3, STORE: 4)
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2
Data variables:
    y        (DATE, STORE, SKU) int64 9 2 2 3 1 6 4 7 6 7 ... 2 5 2 6 6 8 1 1 3
    yhat     (DATE, STORE, SKU) int64 13 3 2 4 0 7 7 7 9 3 ... 0 9 1 2 3 0 0 0 0

As seen above, `xarray` has a very nice html representation of `xarray.Dataset` objects.

Click on the data symbol (the cylinder) to the see the data associated with the `Coordinates` and the `Data`.

You now have one variable (`ds`) which houses the data and the associted meta data. You can also use the `Attributes` for handling things like units. (this is why `xarray` was developed!).

If you would like to know more about `xarray` check out this [overview](http://xarray.pydata.org/en/stable/quick-overview.html).

We can use `xskillscore` on this `xarray.Dataset` via `xarray`'s [Accessor method](http://xarray.pydata.org/en/stable/generated/xarray.register_dataset_accessor.html).

`xskillscore` expects at least 3 arguments for most functions. These are `y`: the target variable; `yhat`: the predicted variable and `dim(s)` the dimension(s) for which to apply the verification metric over.

To replicate the `scikit-learn` metric above, apply RMSE over all the dimensions `[DATE, STORE, SKU]`. RMSE is called `rmse` in xskillscore. #Lastly call `.values` on the object to obtain the data as a `np.array`...

In [13]:
rmse = ds.xs.rmse('y', 'yhat', ['DATE', 'STORE', 'SKU'])
rmse

<xarray.DataArray ()>
array(2.93257566)

If you want just the data from the `xarray.DataArray` you can `.values` on it.

In [14]:
rmse.values

array(2.93257566)

`xskillscore` allows you apply the metric over any combination of dimensions (think of `pandas.groupby.apply` but faster).

For example, your boss has asked you how good are your predictions at store level.

In this case, apply the metrics over the `DATE` and `SKU` dimensions:

In [15]:
rmse = ds.xs.rmse('y', 'yhat', ['DATE', 'SKU'])
rmse

<xarray.DataArray (STORE: 4)>
array([2.17562252, 2.87518115, 3.21455025, 3.32665999])
Coordinates:
  * STORE    (STORE) int64 0 1 2 3

We can use `xarray` a bit further to explore our results.

Let find out which store had the best forecast and which store had the worst forecast:

In [16]:
print('Our forecast performed well for store:')
print(rmse.where(rmse==rmse.min(), drop=True).coords)
print('')
print('Our forecast struggled with store:')
print(rmse.where(rmse==rmse.max(), drop=True).coords)

Our forecast performed well for store:
Coordinates:
  * STORE    (STORE) int64 0

Our forecast struggled with store:
Coordinates:
  * STORE    (STORE) int64 3


# Providing weights to the verification metrics

You can specify weights when calculating skill metrics. Here I will go through an example demonstrating why you may want to apply weights when verifying your forecast.

You boss has asked for you to create a prediction for the next five days. You will update this prediction everyday and there is a larger focus on the performance of the model for the subsequenct day and less of a focus on the fith day.

In this case you can weight your metric so the performance of day 1 has a larger influence than day 5. Here we will apply a linear scaling from 1 to 0 with day 1 having a weight of 1. and day 5 having a weight of 0..

Generate the weights the same size as the `DATE` dimension and put it into an `xarray.DataArray`:

In [17]:
dim = 'DATE'
np_weights = np.linspace(1, 0, num=len(ds[dim]))
weights = xr.DataArray(np_weights, dims=dim)
print(weights)

<xarray.DataArray (DATE: 5)>
array([1.  , 0.75, 0.5 , 0.25, 0.  ])
Dimensions without coordinates: DATE


Now simply add the variable to the `weights` argument: 

In [18]:
ds.xs.rmse('y', 'yhat', 'DATE', weights=weights)

<xarray.DataArray (STORE: 4, SKU: 3)>
array([[3.54964787, 1.        , 1.04880885],
       [2.64575131, 3.50713558, 0.89442719],
       [2.04939015, 2.82842712, 2.56904652],
       [4.01248053, 3.46410162, 3.67423461]])
Coordinates:
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2

and you can compare to the result without the weights:

In [19]:
ds.xs.rmse('y', 'yhat', 'DATE')

<xarray.DataArray (STORE: 4, SKU: 3)>
array([[3.46410162, 1.        , 1.09544512],
       [2.75680975, 4.04969135, 0.89442719],
       [2.44948974, 2.86356421, 4.09878031],
       [3.31662479, 2.82842712, 3.76828874]])
Coordinates:
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2

# Handle missing values

There may be no purchases for certain items in certain stores on certain dates. These entries will be blank in the query from the database.

To mimic data like this create the same type of data structure as before but randomally suppress each row. I have created a simply `if` statement that will drop the row with a probabilty of 0.2 (20%):

In [20]:
random_number_threshold = 0.8

rows = []
for _, date in enumerate(dates):
    for _, store in enumerate(stores):
        for _, sku in enumerate(skus):
            if np.random.rand(1) < random_number_threshold:
                rows.append(
                    dict(
                        {
                            "DATE": date,
                            "STORE": store,
                            "SKU": sku,
                            "QUANTITY_SOLD": np.random.randint(9) + 1,
                        }
                    )
                )
df = pd.DataFrame(rows)
df.rename(columns={"QUANTITY_SOLD": "y"}, inplace=True)
df.set_index(['DATE', 'STORE', 'SKU'], inplace=True)
df.head(10)

y
DATE       STORE SKU   
2020-01-01 0     0    7
                 2    1
           1     0    2
                 1    5
                 2    2
           2     0    6
                 2    4
           3     0    6
                 2    8
2020-01-02 0     0    3

Converting the `pandas.DataFrame` to an `xarray.Dataset` is very handy in this case becuase it will infer the missing entries as `nans` (as long as all indexes are present in the `pandas.DataFrame`):

In [21]:
ds = df.to_xarray()
ds

<xarray.Dataset>
Dimensions:  (DATE: 5, SKU: 3, STORE: 4)
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2
Data variables:
    y        (DATE, STORE, SKU) float64 7.0 nan 1.0 2.0 5.0 ... nan 3.0 3.0 8.0

Click on the data symbol associated with the `y` Data variable to see the `nans`.

You can also use this step in your workflow if simply want to continue working with the `pandas.DataFrame`:

In [22]:
df_with_nans = ds.to_dataframe()
df_with_nans.head(10)

y
DATE       SKU STORE     
2020-01-01 0   0      7.0
               1      2.0
               2      6.0
               3      6.0
           1   0      NaN
               1      5.0
               2      NaN
               3      NaN
           2   0      1.0
               1      2.0

Note: xarray returns the fields alphabetically but it still shows the `nans`.

In most cases you will not know aprior if there will be no purchases for a particular item in a certain store during a day. Therefore, your prediction will not contain `nans` but you would hope the value is low.

Append a prediction column as was done previously:

In [23]:
df_with_nans['yhat'] = df_with_nans['y'] + (df_with_nans['y'] * noise)
df_with_nans.head()

y       yhat
DATE       SKU STORE                
2020-01-01 0   0      7.0  10.446753
               1      2.0   3.411661
               2      6.0   7.584959
               3      6.0   9.522168
           1   0      NaN        NaN

Our prediction contains `nans` so to mimic a realistic prediction replace these with values:

In [24]:
yhat = df_with_nans['yhat']

yhat.loc[pd.isna(yhat)] = yhat[pd.isna(yhat)].apply(lambda x: np.random.randint(9) + 1)

df_with_nans['yhat'] = yhat
df_with_nans.head()

y       yhat
DATE       SKU STORE                
2020-01-01 0   0      7.0  10.446753
               1      2.0   3.411661
               2      6.0   7.584959
               3      6.0   9.522168
           1   0      NaN   2.000000

Now if we try using `scikit-learn`:

In [25]:
mean_squared_error(df_with_nans['y'], df_with_nans['yhat'], squared=False)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

you get a `ValueError: Input contains NaN`.

In `xskillscore` you don't need to worry about this and simply specifiy `skipna=True`:

In [26]:
ds = df_with_nans.to_xarray()
ds.xs.rmse('y', 'yhat', ['DATE', 'STORE', 'SKU'], skipna=True)

<xarray.DataArray ()>
array(3.44001822)

# Handle weights and missing values

You can specifcy weights and skipna together for powerful analysis..

In [27]:
ds.xs.rmse('y', 'yhat', 'DATE', weights=weights, skipna=True)

<xarray.DataArray (SKU: 3, STORE: 4)>
array([[3.03824381, 1.10544282, 1.46657789, 4.96241146],
       [6.44982206, 1.03585395, 2.0144671 , 0.87421379],
       [1.19256817, 2.41157233, 3.45932053, 3.49296654]])
Coordinates:
  * SKU      (SKU) int64 0 1 2
  * STORE    (STORE) int64 0 1 2 3